In [1]:
import pandas as pd
from conllu import parse
from nltk import DependencyGraph
from ufal.udpipe import Model, Pipeline
import re
from transformers import AutoTokenizer
import numpy as np
import random

from modeling import *

In [2]:
tokenizer = BertTokenizer.from_pretrained(pre_trained_model_ckpt)
class_names = ['поступление - перевод', 'общежитие', 'учебная деятельность', 'внеучебная деятельность', 'документы', 'работа', 'финансы']
model = SentimentClassifier(len(class_names))
model.load_state_dict(torch.load('model/best_model_state.bin'))
model = model.to(device)

review_text = "Когда будет сессия?"
encoded_review = tokenizer.encode_plus(review_text, max_length=512, add_special_tokens=True, return_token_type_ids=False, pad_to_max_length=True, return_attention_mask=True,
                                       truncation=True, return_tensors='pt')
input_ids = encoded_review['input_ids'].to(device)
attention_mask=encoded_review['attention_mask'].to(device)
output = model(input_ids, attention_mask)
_,prediction = torch.max(output, dim=1)

print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\ProgramData\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad

Review text: Когда будет сессия?
Sentiment  : учебная деятельность


In [3]:
df = pd.read_excel('make_dataset/08042023_dataset_sort_change_tokenizer_fix.xlsx')
df

,q,a,tq,ta,label
0,"Здраствуйте, если ты уже зачислен на первый ку...",Здравствуйте. Вам необходимо сообщить об этом ...,"[30191., 29508., 34394., 90641., 1523.]","[13802., 29526., 38156.]",0
1,Здравствуйте! В магистратуре можно ли поменять...,Здравствуйте. По данному вопросу обратитесь по...,"[ 7021., 65900., 13552., 13123.]","[ 9168., 77489., 114408.]",0
2,Добрый день! Абитуриент МАГ/261 ФЭН.1Мз. По со...,"Здравствуйте. Приказ уже был, информация о зач...","[20070., 15887., 47518., 7996., 77599., 24706.]","[81373., 15887., 46486., 1761., 24970., 1984.]",0
3,"Здравствуйте, как забрать оригинал аттестата, ...",Здравствуйте. Вам необходимо обратиться в прие...,"[12834., 40597., 63002., 45633., 22451.]","[13802., 29526., 29549., 31283., 15514., 28707.]",0
4,Здравствуйте. Где на сайте можно ознакомиться ...,Здравствуйте. Приказы о зачислении на сайте не...,"[ 7021., 43485.]","[54211., 51623.]",0
...,...,...,...,...,...
47061,Какие программы скидок предлагаются для обуче...,"В зависимости от университета, могут быть пре...","[82877., 11520.]","[ 7603., 39892., 1770., 61692., 11267., 11520....",7
47062,Какова стоимость обучения в университете по м...,Стоимость обучения в университете по медицинс...,"[ 53556, 15850, 12201, 106909, 16753]","[19709., 15850.]",7
47063,Какие дополнительные расходы могут возникнуть...,Дополнительные расходы могут включать в себя ...,"[ 7603., 23000., 51172.]","[ 7603., 23000., 35778., 37744.]",7
47064,Какие программы скидок предлагаются для обуче...,"В зависимости от университета, могут быть пре...","[82877., 11520.]","[ 7603., 39892., 1770., 61692., 11267., 11520....",7


In [4]:
import difflib

def mySort(s1, s2):
  matcher = difflib.SequenceMatcher(None, s1, s2)
  return matcher.ratio()

In [5]:
def get_sov(sent):
    graph = DependencyGraph(tree_str=sent)
    sov = {}
    for triple in graph.triples():
        if triple:
            if triple[0][1] == 'VERB':
                sov[triple[0][0]] = {'subj':'','obj':''}
    for triple in graph.triples():
        if triple:
            if triple[1] == 'nsubj':
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['subj']  = triple[2][0]
            if 'obj' in triple[1]:
                if triple[0][1] == 'VERB':
                    sov[triple[0][0]]['obj'] = triple[2][0]
    return sov

In [6]:
model = Model.load('model/russian-ud-2.0-170801.udpipe')
tokenizer = BertTokenizer.from_pretrained(pre_trained_model_ckpt)
def tokenText(text):
    arrParsed = []
    longParsed = ''

    sent = str(text)
    sent = re.sub("\s\s+", ' ', sent)
    sent = sent.lower()
    sent = sent.strip()
    sent = sent.replace(u'\xa0', u' ')
    sentS = re.split(";|! |\?|\. ", sent)
    sentS = list(filter(None, sentS))

    for nSentS in sentS:
        nSentS = re.sub(r'[^\w\s]','', nSentS) 

        pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, Pipeline.DEFAULT)
        parsed = pipeline.process(nSentS)

        # костыли для dependency graph
        parsed = '\n'.join([line for line in parsed.split('\n') if not line.startswith('#')])
        parsed = parsed.replace('\troot\t', '\tROOT\t')

        if (len(longParsed) < len(parsed)):
            longParsed = parsed

        arrParsed.append(parsed)
    arrParsed = list(filter(None, arrParsed))

    arrSov = []
    for nArrParsed in arrParsed:
        sov = get_sov(nArrParsed)
        arrSov.append(sov)

    bigSov = arrSov[0]
    if (len(arrSov) > 1):
        for i in range(1, len(arrSov)):
            bigSov = { **bigSov, ** arrSov[i] }

    tokenizedA = []
    if (bigSov == {}):
        arrGraph = []
        graph = DependencyGraph(tree_str=longParsed)
        for i in range(0, len(list(graph.triples()))):
            for j in range(0, len(list(graph.triples())[0])):
                if (list(graph.triples())[i][j][1] == 'NOUN'):
                    arrGraph.append(list(graph.triples())[i][j][0])
        arrGraph = set(arrGraph)
        if (len(arrGraph) == 0):
            tokenizedA.append('!');
        else:
            for nArrGraph in arrGraph:
                tokenizedA.append(tokenizer.encode(nArrGraph, add_special_tokens=False))
            tokenizedA = np.concatenate(tokenizedA, axis=0, out=None, dtype=None, casting="same_kind")
    else:
        for k, v in bigSov.items(): 
            tokenizedA.append(tokenizer.encode(k, add_special_tokens=False))
            tokenizedA.append(tokenizer.encode(v['subj'], add_special_tokens=False))
            tokenizedA.append(tokenizer.encode(v['obj'], add_special_tokens=False))
        tokenizedA = np.concatenate(tokenizedA, axis=0, out=None, dtype=None, casting="same_kind")

    return tokenizedA

In [9]:
tokenizer = BertTokenizer.from_pretrained(pre_trained_model_ckpt)
class_names = ['поступление - перевод', 'общежитие', 'учебная деятельность', 'внеучебная деятельность', 'документы', 'работа', 'финансы']
myModel = SentimentClassifier(len(class_names))
myModel.load_state_dict(torch.load('model/best_model_state.bin'))
myModel = myModel.to(device)

text = "Я хочу поступить к вам на психологию?"
encoded_review = tokenizer.encode_plus(text, max_length=512, add_special_tokens=True, return_token_type_ids=False, pad_to_max_length=True, return_attention_mask=True,
                                       truncation=True, return_tensors='pt')
input_ids = encoded_review['input_ids'].to(device)
attention_mask=encoded_review['attention_mask'].to(device)
output = myModel(input_ids, attention_mask)
_,prediction = torch.max(output, dim=1)

tToken = tokenText(text)
tq = []
index = 0
probability = 0.00
textQ = ''
textA = ''
questions = []

for i in range(0, len(df)):
    if (probability <= mySort(tToken, eval(df['tq'][i])) and df['label'][i] == int(prediction)):
        tq = eval(df['tq'][i])
        index = i
        textQ = df['q'][i]
        textA = df['a'][i]
        probability = mySort(tToken, eval(df['tq'][i]))
        questions.append([tq, index, textQ, textA, probability])

max_last_elem = max([que[-1] for que in questions])
max_lists = [lst for lst in questions if lst[-1] == max_last_elem]
rand = random.choice(max_lists)

print(text, '- Текст основного вопроса')
print(class_names[prediction], '- ИИ класс')
print(tToken, '- Токенизированный текст основного вопроса')

print(rand[4]*100, '% - Процент сходства с найденным вопросом')
print(rand[1], '- Номер найденного вопроса')
print(rand[2], '- Текст найденного вопроса')
print(rand[0], '- Токенизированный найденный вопрос')
print(rand[3], '- Ответ к найденному вопросу')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Я хочу поступить к вам на психологию? - Текст основного вопроса
поступление - перевод - ИИ класс
[36349.   877.] - Токенизированный текст основного вопроса
100.0 % - Процент сходства с найденным вопросом
4195 - Номер найденного вопроса
здравствуйте! я хочу поступить к вам на заочное отделение на учёбу, бакалавриат специальность: Менеджмент в туристской и гостиничной индустрии. - Текст найденного вопроса
[36349.0, 877.0] - Токенизированный найденный вопрос
Здравствуйте. Отлично. Рекомендуем начать с изучения Правил приема в ВУЗ на 2020 год. - Ответ к найденному вопросу
